<a href="https://colab.research.google.com/github/victoralcantara75/sickle-cell-classification/blob/main/TCC_2_REAL_OFICIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#math
import numpy as np
import sklearn
import skimage
import sklearn.model_selection

#ts and keras
import tensorflow as tf 
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import applications
from tensorflow.keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

#dimensionality
from sklearn.decomposition import PCA

#classificators
from sklearn.svm import SVC
from sklearn import naive_bayes
from sklearn.ensemble import RandomForestClassifier

#visualization
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

#utils
import os
import imageio
from datetime import datetime
import time

In [ ]:
#dataset
!git clone https://github.com/victoralcantara75/train-test-erythrocytes.git

fatal: destination path 'train-test-erythrocytes' already exists and is not an empty directory.


In [ ]:
#dataset segmented
!git clone https://github.com/victoralcantara75/erythocytes-seg.git

fatal: destination path 'erythocytes-seg' already exists and is not an empty directory.


In [ ]:
classes = ["circular", "falciforme", "outras"]
# classificators = ["svm", "bayes", "forest"]
classificators = ["svm"]
batch = 16
epochs = 30
opt = 'adam'
lr = 0.001

In [ ]:
def loadDir(round):
  train_dir = './train-test-erythrocytes/dataset/5-fold/round_'+str(round)+'/train'
  test_dir = './train-test-erythrocytes/dataset/5-fold/round_'+str(round)+'/test'
  return train_dir, test_dir

In [ ]:
def loadDirSeg(round):
  train_dir = './erythocytes-seg/5-fold/round_'+str(round)+'/train'
  test_dir = './erythocytes-seg/5-fold/round_'+str(round)+'/test'
  return train_dir, test_dir

In [ ]:
def createSaveFile():
  dir = net+"/"
  if not(os.path.isdir(dir)):
    os.mkdir(dir)

  today = datetime.today()
  path = dir + str(today) + ".txt" 

  saveFile = open(path, 'a')
  return saveFile

In [ ]:
def createModel(net):

  if net == "mobile":
    base_model = applications.MobileNet(weights='imagenet', include_top=True)
    vector = base_model.get_layer("reshape_2").output
  elif net == "resnet":
    base_model = applications.ResNet50(weights='imagenet', include_top=True)
    vector = base_model.get_layer("avg_pool").output
  elif net == "dense":
    base_model = applications.DenseNet169(weights='imagenet', include_top=True)
    vector = base_model.get_layer("avg_pool").output

  model = tf.keras.Model(base_model.input, vector)
  return model

In [ ]:
def toArray(X_list_train, Y_list_train, X_list_test, Y_list_test):
  train_imgs = np.asarray(X_list_train, dtype=np.float32)
  train_labels = np.asarray(Y_list_train, dtype=np.float32)

  test_imgs = np.asarray(X_list_test, dtype=np.float32)
  test_labels = np.asarray(Y_list_test, dtype=np.float32)

  return train_imgs, train_labels, test_imgs, test_labels


In [ ]:
def extract_features_test(path, model):
  print('extracting features')
  x_list = []
  y_list = []

  for label in range(3):    
    folder_path = os.path.join(path, classes[label])
    for file in os.listdir(folder_path):    
        file_path = os.path.join(folder_path, file)
        
        if dataset == "seg" and not(file.endswith(".tif")):
            continue
        
        # load image
        img = image.load_img(file_path, target_size=(224,224))
        # convert image to numpy array
        img_arr = image.img_to_array(img)
        # add 1 more dimension
        img_arr_b = np.expand_dims(img_arr, axis=0)
        # preprocess image
        input_img = preprocess_input(img_arr_b)
        # extract feature
        features = model.predict(input_img)
        x_list.append(features.ravel())
        y_list.append(label)

  return x_list, y_list

In [ ]:
def extract_features_train(path, model):
  print('extracting features')
  x_list = []
  y_list = []

  for label in range(3):    
    folder_path = os.path.join(path, classes[label])
    for file in os.listdir(folder_path):    
        file_path = os.path.join(folder_path, file)
        
        if dataset == "seg" and not(file.endswith(".tif")):
            continue
        
        # load image
        img = image.load_img(file_path, target_size=(224,224))
        # convert image to numpy array
        img_arr = image.img_to_array(img)
        # add 1 more dimension
        img_arr_b = np.expand_dims(img_arr, axis=0)
        # preprocess image
        input_img = preprocess_input(img_arr_b)
        #data augmentation
        da = []
        img_vertical_flip = np.flipud(input_img)
        img_horizontal_flip = np.fliplr(input_img)
        da.append(input_img)
        da.append(img_vertical_flip)
        da.append(img_horizontal_flip)
        # extract feature
        for data in da:
          features = model.predict(data)
          x_list.append(features.ravel())
          y_list.append(label)

        # VISUALIZAR EXEMPLOS DOS MAPAS DE CARACTERISTICAS
        # index = 5
        # for i in range(0, 12):
        #   viewModel = Model(inputs=model.inputs, outputs=model.layers[i].output)
        #   viewFeatures = viewModel.predict(input_img)
        #   plt.imshow(viewFeatures[0, :, :, 1], cmap='gray')
        #   plt.show()
        #   index += 10
 
          

  return x_list, y_list

In [ ]:
def reduct_features(imgs):
  print('reducting features')
  pca = PCA(n_components=3)
  pca.fit(imgs)
  reduc_features = pca.transform(imgs)
  return reduc_features


In [ ]:
def classificate(name, clf, accs, train_features, train_labels, test_features, test_labels, saveFile):
  clf.fit(train_features, train_labels)
  preds = clf.predict(test_features)

  report = classification_report(test_labels, preds, target_names=classes, output_dict=True)
  accs.append(report['accuracy'])
  print("Accuracy: ", report['accuracy'])
  saveFile.write(name + '\n')
  saveFile.write(classification_report(test_labels, preds, target_names=classes, digits=4))
  return accs

In [ ]:
def svm_grid_search(C, kernel, train_X, train_Y):
    accuracy_score_list = []
    
    for c in C:
        # Model training
        svmClassifier = SVC(C = c, kernel = kernel)
        svmClassifier.fit(train_X, train_Y.ravel())
        # Prediction on test set
        pred_y = svmClassifier.predict(train_X)
        # Accuracy
        accuracy = accuracy_score(train_Y, pred_y)
        accuracy_score_list.append(accuracy)
        print('Regularization parameters: ', c, 'Accuracy', accuracy)
    
    max_accurarcy_id = accuracy_score_list.index(max(accuracy_score_list))
    return C[max_accurarcy_id] 

In [ ]:
net = input("CNN (mobile, dense, resnet):")
dataset = input("Data (seg, original): ")

accsSVM = []
accsBayes = []
accsForest = []
model = createModel(net)

saveFile = createSaveFile()

start = time.time()
for i in range (1, 6):
  
  print("ROUND ", i)

  if dataset == "original":
    train_dir, test_dir = loadDir(i)
  elif dataset == "seg":
    train_dir, test_dir = loadDirSeg(i)

  X_list_train, Y_list_train = extract_features_train(train_dir, model)
  X_list_test, Y_list_test = extract_features_test(test_dir, model)

  train_imgs, train_labels, test_imgs, test_labels = toArray(X_list_train, Y_list_train, X_list_test, Y_list_test)

  print("train/test images shape")
  print(train_imgs.shape)
  print(test_imgs.shape)

  # reduc_features_train = reduct_features(train_imgs)
  # reduc_features_test = reduct_features(test_imgs)
  # C, kernel = [0.1 * i for i in range(1, 30)], 'linear'
  # opt_c = svm_grid_search(C, kernel, reduc_features_train, train_labels)
  # clf = SVC(C= opt_c, kernel= 'linear')

  for clfs in classificators:

    if clfs == "svm":
      clf = SVC(C = 2.9, kernel= 'linear')
      classificate(clfs, clf, accsSVM, train_imgs, train_labels, test_imgs, test_labels, saveFile)
    # if clfs == "bayes":
    #   clf = naive_bayes.GaussianNB()
    #   classificate(clfs, clf, accsBayes, train_imgs, train_labels, test_imgs, test_labels, saveFile)
    # if clfs == "forest":
    #   clf = RandomForestClassifier()
    #   classificate(clfs, clf, accsForest, train_imgs, train_labels, test_imgs, test_labels, saveFile)
end = time.time()

print("Tempo - ", end - start)
print("Result SVM: ", np.mean(accsSVM))
# print("Result Bayes: ", np.mean(accsBayes))
# print("Result Random Forest: ", np.mean(accsForest))

saveFile.write("Result SVM: "+ str(np.mean(accsSVM)))
# saveFile.write("\n Result Bayes: "+ str(np.mean(accsBayes)))
# saveFile.write("\n Result Forest: "+ str(np.mean(accsBayes)))

saveFile.close()

CNN (mobile, dense, resnet):mobile
Data (seg, original): seg
ROUND  1
extracting features
extracting features
train/test images shape
(1506, 1000)
(124, 1000)
Accuracy:  0.9596774193548387
ROUND  2
extracting features
extracting features
train/test images shape
(1500, 1000)
(126, 1000)
Accuracy:  0.9682539682539683
ROUND  3
extracting features
extracting features
train/test images shape
(1500, 1000)
(126, 1000)
Accuracy:  0.9841269841269841
ROUND  4
extracting features
extracting features
train/test images shape
(1500, 1000)
(126, 1000)
Accuracy:  0.9682539682539683
ROUND  5
extracting features
extracting features
train/test images shape
(1506, 1000)
(124, 1000)
Accuracy:  0.9596774193548387
Tempo -  287.9055218696594
Result SVM:  0.9679979518689196
